In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data_all_small_3.csv', index_col=0, encoding='utf-8')

In [3]:
data.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,...,Remodeledyear,Heating,Cooling,Parking,lotsize,Pricesqft,Pricesqft2,rating1,Distance1,Grades1
0,Active,No,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,...,'','Central A/C,'','',0.0,'144',144.0,'4',2.70,'3–5'
1,for sale,No,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",...,'','','','',5828.0,'159',159.0,'4/10',1.01,'PK-8'
2,for sale,No,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",...,'1967','Forced Air','Central','Attached Garage',8.0,'965',965.0,'8/10',1.19,'6-8'
3,for sale,No,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",...,'2006','Forced Air','Central','Detached Garage',8.0,'371',371.0,'9/10',0.10,'PK-4'
4,for sale,No,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,...,'','','','',10.0,None,0.0,'5/10',3.03,'PK-6'


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377185 entries, 0 to 377184
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   status         337267 non-null  object 
 1   private pool   377185 non-null  object 
 2   propertyType   342452 non-null  object 
 3   street         377183 non-null  object 
 4   baths          270847 non-null  object 
 5   homeFacts      377185 non-null  object 
 6   fireplace      103115 non-null  object 
 7   city           377151 non-null  object 
 8   schools        377185 non-null  object 
 9   sqft           336608 non-null  object 
 10  zipcode        377185 non-null  object 
 11  beds           285903 non-null  object 
 12  state          377185 non-null  object 
 13  stories        226470 non-null  object 
 14  mls-id         24942 non-null   object 
 15  PrivatePool    377185 non-null  object 
 16  MlsId          310305 non-null  object 
 17  target         374704 non-nul

In [5]:
# 'beds', 'stories' , 'fireplace', 'street' пока удалим дальше поймем как их разбирать
data.drop(['schools', 'status','propertyType','baths','homeFacts','fireplace','sqft','target','beds'], axis='columns', inplace=True)
data.drop(['street'], axis='columns', inplace=True)
data.drop(['stories','mls-id','MlsId', 'Pricesqft'], axis='columns', inplace=True)



Необходимо еще обработать данные

In [6]:
def replace1(x):
   x = str(x).replace("'" , '')
   x = str(x).replace(" " , '')
   return x

In [7]:
data['Yearbuilt'] = data.apply(lambda x: replace1(x['Yearbuilt']), axis = 1)
data['Remodeledyear'] = data.apply(lambda x: replace1(x['Remodeledyear']), axis = 1)
data['Distance1'] = data.apply(lambda x: replace1(x['Distance1']), axis = 1)
#data['Distance2'] = data.apply(lambda x: replace1(x['Distance2']), axis = 1)
data['lotsize'] = data.apply(lambda x: replace1(x['lotsize']), axis = 1)
data['baths2'] = data.apply(lambda x: replace1(x['baths2']), axis = 1)
data['sqft2'] = data.apply(lambda x: replace1(x['sqft2']), axis = 1)

In [8]:
def replace_to(x):
   x = str(x).replace("'" , '')
   x = str(x).replace("to" , '-')
   x = str(x).replace(" " , '')
   x = str(x).replace("nan" , '0')
   x = str(x).replace("None" , '0')
   x = str(x).replace("NR" , '0')
   x = str(x).replace("NA" , '0')
    
   return x

In [9]:
data['Grades1'] = data.apply(lambda x: replace_to(x['Grades1']), axis = 1)
data['rating1'] = data.apply(lambda x: replace_to(x['rating1']), axis = 1)

In [10]:
def replace_city(x):
   x = str(x).replace("'" , '')
   x = str(x).replace(" " , '')
   x = str(x).replace("." , '')
   x = str(x).replace("-" , '')
   x = str(x).upper()
   х = str(x).strip() 
   return x

In [11]:
data['city'] = data.apply(lambda x: replace_city(x['city']), axis = 1)

In [12]:
#  В некоторых строках стоит 27713-6702   возьмем только первую часть
def replace_zip(x):
   x = str(x).split("-")[0]
   return x

In [13]:
data['zipcode'] = data.apply(lambda x: replace_zip(x['zipcode']), axis = 1)

In [14]:
# Удалим грязные данные
idx = (data.loc[data['city']==''])
data.drop(idx.index, inplace=True)

idx = (data.loc[data['rating1']==''])
data.loc[idx.index, 'rating1'] ='0'

idx = (data.loc[data['sqft2']=='NAN'])
data.loc[idx.index, 'sqft2'] ='0' 
idx = (data.loc[data['sqft2']=='--'])
data.loc[idx.index, 'sqft2'] ='0' 

idx = (data.loc[data['Distance1']=='nan'])
data.loc[idx.index, 'Distance1'] ='0' 

idx = (data.loc[data['Remodeledyear']=='0'])
data.loc[idx.index, 'Remodeledyear'] ='NoData'    
idx = (data.loc[data['Remodeledyear']==''])
data.loc[idx.index, 'Remodeledyear'] ='NoData'
idx = (data.loc[data['Remodeledyear']=='None'])
data.loc[idx.index, 'Remodeledyear'] ='NoData'

idx = (data.loc[data['Yearbuilt']=='559990649990'])
data.drop(idx.index, inplace=True)
idx = (data.loc[data['Yearbuilt']=='1'])
data.drop(idx.index, inplace=True)

idx = (data.loc[data['Yearbuilt']=='None'])
data.loc[idx.index, 'Yearbuilt'] ='NoData'
idx = (data.loc[data['Yearbuilt']==''])
data.loc[idx.index, 'Yearbuilt'] ='NoData'

idx = (data.loc[data['target2']=='NAN'])
data.drop(idx.index, inplace=True)
idx = (data.loc[data['target2']=='1'])
data.drop(idx.index, inplace=True)
idx = (data.loc[data['target2']=='1215 - 1437'])
data.drop(idx.index, inplace=True)

In [15]:
data.columns

Index(['private pool', 'city', 'zipcode', 'state', 'PrivatePool', 'baths2',
       'sqft2', 'target2', 'status2', 'propertyType2', 'Yearbuilt',
       'Remodeledyear', 'Heating', 'Cooling', 'Parking', 'lotsize',
       'Pricesqft2', 'rating1', 'Distance1', 'Grades1'],
      dtype='object')

In [ ]:
# сохраним итоговый файл
data.to_csv('data_all_small_result.csv')

Построим первую модель

In [16]:
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder 

In [17]:
X = data.drop(['target2'], axis=1,)
y = data.target2.values.astype('int64')

In [19]:
# Колонки в которых находяться категориальные данные преобразуем через get_dummies
# Колонки в которых находяться числовые данные приведем  к типу float
def _prepare(df_input):
    df_output = df_input.copy()
    #labelencoder = LabelEncoder()
    dicts = {}
    _column =['Yearbuilt','Remodeledyear','Heating','Cooling', 'Parking','rating1', #'street'
              'Grades1','propertyType2', 'status2','city','state','zipcode','PrivatePool','private pool']
    
    for _name in _column:
           #labelencoder.fit(df_output[_name].drop_duplicates())
           #dicts[_name] = list(labelencoder.classes_)
           #df_output[_name] = labelencoder.transform(df_output[_name])
           df_output = pd.get_dummies(df_output, columns=[_name], prefix=[_name+"_is",] ) 
    
    _column =['sqft2','baths2','lotsize','Pricesqft2','Distance1']
    for _name in _column:
            #print(_name)
            df_output[_name] = df_output[_name].astype(float)
    
    return df_output

In [20]:
X1 = _prepare(X) # преобразуем для работы с моделью

In [21]:
X1.head()

,baths2,sqft2,lotsize,Pricesqft2,Distance1,Yearbuilt_is_1019,Yearbuilt_is_1057,Yearbuilt_is_1060,Yearbuilt_is_1208,Yearbuilt_is_1700,...,zipcode_is_99218,zipcode_is_99223,zipcode_is_99224,zipcode_is_99336,zipcode_is_99337,zipcode_is_99338,PrivatePool_is_No,PrivatePool_is_Yes,private pool_is_No,private pool_is_Yes
0,0.0,2900.0,0.0,144.0,2.70,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,3.0,1947.0,5828.0,159.0,1.01,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,2.0,3000.0,8.0,965.0,1.19,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,8.0,6457.0,8.0,371.0,0.10,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,0.0,0.0,10.0,0.0,3.03,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X1, y, test_size=0.3, random_state=42, shuffle=True)

При попытке построить ниже описанную модель получаю ошибку:
MemoryError: Unable to allocate 27.9 GiB for an array with shape (9993, 374655) and data type float64


In [ ]:
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X1)

model = DecisionTreeRegressor(
 #   n_estimators=400,
 #   max_depth=26,
 #   max_features=0.5,
 #   min_samples_leaf=1,
 #   min_samples_split=5,
    random_state=42
 #   n_jobs=-1
)
model.fit(X_scaled, y)

#scores = cross_val_score(model, X_scaled, y, scoring='r2', cv=10, n_jobs=-1)
#print(f'cv mean: {np.round(np.mean(scores), 4)}')
#print(f'cv std:  {np.round(np.std(scores), 4)}')

y_pred_train = model_dict[model].predict(X_train_scaled)
y_pred_valid = model_dict[model].predict(X_valid_scaled)

score_train = r2_score(y_train, y_pred_train)
score_valid = r2_score(y_valid, y_pred_valid)

print(f'{model}: \n\ttrain: {np.round(score_train, 4)} \n\tvalid: {np.round(score_valid, 4)}')